<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix, precision_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv('/datasets/Churn.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/Churn.csv'

## Исследование задачи

In [ ]:
df.head()

RowNumber — индекс строки в данных \
CustomerId — уникальный идентификатор клиента \
Surname — фамилия \
CreditScore — кредитный рейтинг \
Geography — страна проживания \
Gender — пол \
Age — возраст \
Tenure — сколько лет человек является клиентом банка \
Balance — баланс на счёте \
NumOfProducts — количество продуктов банка, используемых клиентом \
HasCrCard — наличие кредитной карты \
IsActiveMember — активность клиента \
EstimatedSalary — предполагаемая зарплата 

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
a = df['Tenure'].median()
a

In [ ]:
df['Tenure'].mean()

In [ ]:
df['Tenure'] = df['Tenure'].fillna(a)

Так как медиана является более устойчивым показателем к выбросам в данных. В нашем случае медиана (=5) практически равна среднему значению (=4.997690023099769). Поэтому можем предположить, что данные имеют нормальное распределение.  

In [ ]:
df.info()

In [ ]:
df['Geography'].unique()

In [ ]:
dfp = df.drop(['RowNumber','CustomerId', 'Surname'], axis = 1)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
encoder = OrdinalEncoder()
encoder.fit(dfp[['Geography', 'Gender']])
dfp[['Geography', 'Gender']] = encoder.transform(dfp[['Geography', 'Gender']])
dfp = pd.DataFrame(dfp, columns = dfp.columns)

In [ ]:
dfp

In [ ]:
y = dfp['Exited']
x = dfp.drop(['Exited'], axis = 1)

In [ ]:
x_train, x_v, y_train, y_v = train_test_split(x,y,test_size = 0.4, random_state = 12345)
x_valid, x_test, y_valid, y_test = train_test_split(x_v,y_v,test_size = 0.5, random_state = 12345)

In [ ]:
best_score = 0
best_depth = 0
best_est = 0
for est in range(5,70,5):
    for depth in range(3,25):
        model = RandomForestClassifier(max_depth = depth, n_estimators = est, random_state = 12345)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_valid)
        sc = accuracy_score(y_valid, y_pred)  
        if sc > best_score:
            best_score = sc
            best_depth = depth
            best_est = est
print(best_depth)
print(best_est)
print(best_score)

## Борьба с дисбалансом

In [ ]:
y.value_counts(normalize=1)

In [ ]:
y.plot(kind = 'hist', bins = 2)

In [ ]:
def score(y_valid,y_pred):
    print('Полнота', recall_score(y_valid, y_pred))
    print('Точность', precision_score(y_valid, y_pred))
    print('F1', f1_score(y_valid, y_pred))

In [ ]:
def model(x_train, y_train, x_valid, y_valid):
    model_DT = DecisionTreeClassifier(random_state=12345)
    model_DT.fit(x_train, y_train)
    DT = pd.Series(model_DT.predict(x_valid)).value_counts(normalize=1)
    
    model_RF = RandomForestClassifier(random_state=12345)
    model_RF.fit(x_train, y_train)
    RF = pd.Series(model_RF.predict(x_valid)).value_counts(normalize=1)

    model_LG = LogisticRegression(solver='liblinear',random_state=12345)
    model_LG.fit(x_train, y_train)
    LG = pd.Series(model_LG.predict(x_valid)).value_counts(normalize=1)
    
    print('Доли ответов для дерева решений')
    print(DT)
    print('Доли ответов для случайного леса')
    print(RF)
    print('Доли ответов для логистической регрессии')
    print(LG)

In [ ]:
model(x_train, y_train, x_valid, y_valid)

In [ ]:
model_RF = RandomForestClassifier(max_depth = 10, n_estimators = 40, random_state=12345)
model_RF.fit(x_train, y_train)
y_RF = model_RF.predict(x_valid)
confusion_matrix(y_valid, y_RF)

In [ ]:
score(y_valid, y_RF)

In [ ]:
probabilities_valid = model_RF.predict_proba(x_valid)
probabilities_one_valid = probabilities_valid[:, 1]

In [ ]:
roc = model_RF.predict_proba(x_valid)[:,1]
auc_roc = roc_auc_score(y_valid, roc)
auc_roc

In [ ]:
fpr, tpr, thresholds = roc_curve(y_valid, probabilities_one_valid) 
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC-кривая")
plt.show()

In [ ]:
def upsample(x,y,repeat):
    x_zeros = x[y==0]
    x_ones = x[y==1]
    y_zeros = y[y==0]
    y_ones = y[y==1]
    
    x_upsampled = pd.concat([x_zeros] + [x_ones]*repeat)
    y_upsampled = pd.concat([y_zeros] + [y_ones]*repeat)
    x_upsampled, y_upsampled  = shuffle(x_upsampled, y_upsampled, random_state=12345)
    
    return x_upsampled, y_upsampled

In [ ]:
x_upsampled, y_upsampled = upsample(x_train,y_train,4)

In [ ]:
y_upsampled.value_counts(normalize=1)

In [ ]:
model_RF_1 = RandomForestClassifier(max_depth = 10, n_estimators = 40, random_state=12345)
model_RF_1.fit(x_upsampled, y_upsampled)
y_RF_1 = model_RF_1.predict(x_valid)
confusion_matrix(y_valid, y_RF_1)

In [ ]:
score(y_valid, y_RF_1)

In [ ]:
def downsample(x,y,fraction):
    x_zeros = x[y==0]
    x_ones = x[y==1]
    y_zeros = y[y==0]
    y_ones = y[y==1]
    
    x_downsample = pd.concat([x_zeros.sample(frac=fraction, random_state=12345)]+[x_ones])
    y_downsample = pd.concat([y_zeros.sample(frac=fraction, random_state=12345)]+[y_ones])
    x_downsample,y_downsample = shuffle(x_downsample,y_downsample,random_state=12345)
    
    return x_downsample,y_downsample

In [ ]:
x_downsample,y_downsample = downsample(x_train,y_train,0.25)

In [ ]:
y_downsample.value_counts(normalize=1)

In [ ]:
model_RF = RandomForestClassifier(max_depth = 15, n_estimators = 30, random_state=12345)
model_RF.fit(x_downsample,y_downsample)
y_RF = model_RF.predict(x_valid)
confusion_matrix(y_valid, y_RF)

In [ ]:
score(y_valid, y_RF)

Было рассмотрено три модели для решения задачи: решающее дерево, случайный лес, логистическая регрессия. Для дальнейшего использования я оставила модель случайного леса, так как эта модель лучше себя показала при тестировании. Далее были подобраны оптимальные гиперпараметры. И были рассмотрены два метода борьбы с дисбалансом классов. Метод увеличения выборки проявил себя лучше. Поэтому мы оставили его. Тестирование происходило модели случайного леса с подобранными гиперпараметрами и метода увеличения выборки для борьбы с дисбалансом классов. В результате мы добились нужного нам результата - F1 > 0.59

## Тестирование модели

In [ ]:
y_pred = model_RF_1.predict(x_test)
score(y_test, y_pred)

In [ ]:
probabilities_test = model_RF.predict_proba(x_test)
probabilities_one_test = probabilities_test[:, 1]

In [ ]:
roc = model_RF_1.predict_proba(x_test)[:,1]
auc_roc = roc_auc_score(y_test, roc)
auc_roc

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, probabilities_one_test) 
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC-кривая")
plt.show()

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*